# Diffusion Models for Predictive Maintenance

This notebook implements **Diffusion Models** for condition monitoring:

1. **Denoising Diffusion for Anomaly Detection** - Reconstruction-based detection
2. **Synthetic Fault Generation** - Data augmentation for rare faults
3. **Time Series Imputation** - Fill missing sensor data

## Why Diffusion Models for Predictive Maintenance?

| Advantage | Description |
|-----------|-------------|
| **High-Quality Generation** | Generate realistic synthetic fault data |
| **Anomaly Detection** | Denoising error reveals anomalies |
| **Few-Shot Learning** | Learn from limited fault examples |
| **Uncertainty Quantification** | Probabilistic predictions |

## Diffusion Process

```
Forward (add noise):   x_t = √(ᾱ_t) · x_0 + √(1-ᾱ_t) · ε
Reverse (denoise):     x_{t-1} = model(x_t, t) 
Anomaly score:         ||x_0 - denoise(x_0 + noise)||
```

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, classification_report
import seaborn as sns
import os
import json

np.random.seed(42)

# Check TensorFlow availability
try:
    import tensorflow as tf
    from tensorflow import keras
    from tensorflow.keras import layers
    print(f"TensorFlow {tf.__version__} available")
    HAS_TF = True
except ImportError:
    print("TensorFlow not available")
    HAS_TF = False

# Output directories
DATA_DIR = '../data/diffusion'
MODEL_DIR = '../models/diffusion'
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

print("Setup complete!")

## 1. Diffusion Model Building Blocks

### 1.1 Noise Schedule

Define how noise is added during the forward process.

In [ ]:
def get_noise_schedule(T=1000, schedule='linear'):
    """
    Generate noise schedule for diffusion.
    
    Args:
        T: Number of diffusion steps
        schedule: 'linear' or 'cosine'
        
    Returns:
        betas, alphas, alpha_bar (cumulative product)
    """
    if schedule == 'linear':
        beta_start = 1e-4
        beta_end = 0.02
        betas = np.linspace(beta_start, beta_end, T)
        
    elif schedule == 'cosine':
        # Cosine schedule from "Improved Denoising Diffusion"
        s = 0.008
        steps = np.arange(T + 1)
        f = np.cos((steps / T + s) / (1 + s) * np.pi / 2) ** 2
        alpha_bar = f / f[0]
        betas = 1 - alpha_bar[1:] / alpha_bar[:-1]
        betas = np.clip(betas, 0.0001, 0.999)
    
    alphas = 1 - betas
    alpha_bar = np.cumprod(alphas)
    
    return betas.astype(np.float32), alphas.astype(np.float32), alpha_bar.astype(np.float32)

# Visualize schedules
T = 200  # Fewer steps for time series (vs 1000 for images)

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for schedule in ['linear', 'cosine']:
    betas, alphas, alpha_bar = get_noise_schedule(T, schedule)
    
    axes[0].plot(betas, label=schedule)
    axes[1].plot(alphas, label=schedule)
    axes[2].plot(alpha_bar, label=schedule)

axes[0].set_title('Beta (noise added)')
axes[1].set_title('Alpha (signal retained)')
axes[2].set_title('Alpha Bar (cumulative signal)')

for ax in axes:
    ax.legend()
    ax.set_xlabel('Timestep t')
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(f'{DATA_DIR}/noise_schedule.png', dpi=150, bbox_inches='tight')
plt.show()

In [ ]:
if HAS_TF:
    
    class SinusoidalPositionEmbeddings(layers.Layer):
        """
        Sinusoidal embeddings for diffusion timestep.
        """
        
        def __init__(self, dim, **kwargs):
            super().__init__(**kwargs)
            self.dim = dim
            
        def call(self, timesteps):
            half_dim = self.dim // 2
            embeddings = np.log(10000) / (half_dim - 1)
            embeddings = tf.exp(tf.range(half_dim, dtype=tf.float32) * -embeddings)
            embeddings = tf.cast(timesteps, tf.float32)[:, None] * embeddings[None, :]
            embeddings = tf.concat([tf.sin(embeddings), tf.cos(embeddings)], axis=-1)
            return embeddings
        
        def get_config(self):
            config = super().get_config()
            config.update({'dim': self.dim})
            return config
    
    print("SinusoidalPositionEmbeddings defined")

In [ ]:
if HAS_TF:
    
    class ResidualBlock1D(layers.Layer):
        """
        1D Residual block for time series denoising.
        """
        
        def __init__(self, channels, kernel_size=3, **kwargs):
            super().__init__(**kwargs)
            self.channels = channels
            self.kernel_size = kernel_size
            
        def build(self, input_shape):
            self.conv1 = layers.Conv1D(self.channels, self.kernel_size, padding='same')
            self.conv2 = layers.Conv1D(self.channels, self.kernel_size, padding='same')
            self.norm1 = layers.LayerNormalization()
            self.norm2 = layers.LayerNormalization()
            self.time_proj = layers.Dense(self.channels)
            
            # Skip connection if channels change
            in_channels = input_shape[-1]
            if in_channels != self.channels:
                self.skip_conv = layers.Conv1D(self.channels, 1)
            else:
                self.skip_conv = None
                
            super().build(input_shape)
            
        def call(self, inputs, time_emb=None):
            x = inputs
            
            # First conv
            h = self.norm1(x)
            h = tf.nn.silu(h)
            h = self.conv1(h)
            
            # Add time embedding
            if time_emb is not None:
                time_emb = self.time_proj(tf.nn.silu(time_emb))
                h = h + time_emb[:, None, :]
            
            # Second conv
            h = self.norm2(h)
            h = tf.nn.silu(h)
            h = self.conv2(h)
            
            # Skip connection
            if self.skip_conv is not None:
                x = self.skip_conv(x)
            
            return x + h
        
        def get_config(self):
            config = super().get_config()
            config.update({'channels': self.channels, 'kernel_size': self.kernel_size})
            return config
    
    print("ResidualBlock1D defined")

In [ ]:
if HAS_TF:
    
    def build_denoising_model(seq_length, n_features, time_embed_dim=64, channels=[64, 128, 64]):
        """
        Build U-Net style denoising model for 1D time series.
        
        Predicts the noise ε given noisy input x_t and timestep t.
        """
        # Inputs
        x_input = keras.Input(shape=(seq_length, n_features), name='x_noisy')
        t_input = keras.Input(shape=(), dtype=tf.int32, name='timestep')
        
        # Time embedding
        time_emb = SinusoidalPositionEmbeddings(time_embed_dim)(t_input)
        time_emb = layers.Dense(time_embed_dim * 2, activation='silu')(time_emb)
        time_emb = layers.Dense(time_embed_dim)(time_emb)
        
        # Initial projection
        x = layers.Conv1D(channels[0], 3, padding='same')(x_input)
        
        # Encoder (downsampling)
        skips = []
        for i, ch in enumerate(channels[:-1]):
            x = ResidualBlock1D(ch)(x, time_emb)
            x = ResidualBlock1D(ch)(x, time_emb)
            skips.append(x)
            # Downsample
            x = layers.Conv1D(ch, 3, strides=2, padding='same')(x)
        
        # Middle
        x = ResidualBlock1D(channels[-1])(x, time_emb)
        x = ResidualBlock1D(channels[-1])(x, time_emb)
        
        # Decoder (upsampling)
        for i, ch in enumerate(reversed(channels[:-1])):
            # Upsample
            x = layers.UpSampling1D(2)(x)
            # Crop or pad to match skip size
            skip = skips[-(i+1)]
            x = layers.Conv1D(ch, 3, padding='same')(x)
            # Resize if needed
            if x.shape[1] != skip.shape[1]:
                x = tf.image.resize(x[:, :, None, :], [skip.shape[1], 1])[:, :, 0, :]
            x = layers.Concatenate()([x, skip])
            x = ResidualBlock1D(ch)(x, time_emb)
            x = ResidualBlock1D(ch)(x, time_emb)
        
        # Output: predict noise
        x = layers.LayerNormalization()(x)
        x = tf.nn.silu(x)
        output = layers.Conv1D(n_features, 3, padding='same')(x)
        
        # Resize to exact input shape if needed
        if output.shape[1] != seq_length:
            output = tf.image.resize(output[:, :, None, :], [seq_length, 1])[:, :, 0, :]
        
        model = keras.Model(inputs=[x_input, t_input], outputs=output)
        return model
    
    print("Denoising model builder defined")

## 2. Generate Training Data

In [ ]:
def generate_sensor_data(n_samples=1000, seq_length=128, n_features=4):
    """
    Generate sensor time series data with normal and anomalous patterns.
    """
    X_normal = []
    X_anomaly = []
    
    # Normal samples (majority)
    for _ in range(n_samples):
        t = np.linspace(0, 4 * np.pi, seq_length)
        
        # Multi-sensor normal behavior
        features = np.zeros((seq_length, n_features))
        
        # Feature 0: Vibration (periodic with small noise)
        freq = np.random.uniform(0.8, 1.2)
        features[:, 0] = np.sin(freq * t) + np.random.normal(0, 0.1, seq_length)
        
        # Feature 1: Temperature (slowly varying)
        base_temp = np.random.uniform(0.4, 0.6)
        features[:, 1] = base_temp + 0.1 * np.sin(0.2 * t) + np.random.normal(0, 0.02, seq_length)
        
        # Feature 2: Pressure (stable with noise)
        features[:, 2] = 0.5 + np.random.normal(0, 0.05, seq_length)
        
        # Feature 3: Current (correlated with vibration)
        features[:, 3] = 0.3 * features[:, 0] + 0.5 + np.random.normal(0, 0.08, seq_length)
        
        X_normal.append(features)
    
    # Anomalous samples (minority - for testing)
    n_anomaly = n_samples // 5
    
    for _ in range(n_anomaly):
        t = np.linspace(0, 4 * np.pi, seq_length)
        features = np.zeros((seq_length, n_features))
        
        anomaly_type = np.random.choice(['spike', 'drift', 'noise', 'flatline'])
        
        # Start with normal pattern
        freq = np.random.uniform(0.8, 1.2)
        features[:, 0] = np.sin(freq * t) + np.random.normal(0, 0.1, seq_length)
        features[:, 1] = 0.5 + 0.1 * np.sin(0.2 * t) + np.random.normal(0, 0.02, seq_length)
        features[:, 2] = 0.5 + np.random.normal(0, 0.05, seq_length)
        features[:, 3] = 0.3 * features[:, 0] + 0.5 + np.random.normal(0, 0.08, seq_length)
        
        # Apply anomaly
        if anomaly_type == 'spike':
            # Random spikes
            spike_locs = np.random.choice(seq_length, size=np.random.randint(3, 8), replace=False)
            features[spike_locs, 0] += np.random.uniform(2, 4, len(spike_locs))
            
        elif anomaly_type == 'drift':
            # Gradual drift
            drift = np.linspace(0, np.random.uniform(0.5, 1.5), seq_length)
            features[:, 1] += drift
            
        elif anomaly_type == 'noise':
            # Increased noise
            features[:, 0] += np.random.normal(0, 0.8, seq_length)
            features[:, 2] += np.random.normal(0, 0.3, seq_length)
            
        elif anomaly_type == 'flatline':
            # Sensor stuck
            start = np.random.randint(0, seq_length // 2)
            features[start:, 2] = features[start, 2]
        
        X_anomaly.append(features)
    
    return np.array(X_normal), np.array(X_anomaly)

# Generate data
print("Generating sensor data...")
X_normal, X_anomaly = generate_sensor_data(n_samples=1500, seq_length=128, n_features=4)
print(f"Normal samples: {X_normal.shape}")
print(f"Anomaly samples: {X_anomaly.shape}")

In [ ]:
# Visualize samples
fig, axes = plt.subplots(2, 4, figsize=(16, 6))

# Normal samples
for i in range(4):
    axes[0, i].plot(X_normal[i, :, 0], label='Vibration', alpha=0.7)
    axes[0, i].plot(X_normal[i, :, 1], label='Temp', alpha=0.7)
    axes[0, i].set_title(f'Normal Sample {i+1}')
    if i == 0:
        axes[0, i].legend(fontsize=8)

# Anomaly samples
for i in range(4):
    axes[1, i].plot(X_anomaly[i, :, 0], label='Vibration', alpha=0.7)
    axes[1, i].plot(X_anomaly[i, :, 1], label='Temp', alpha=0.7)
    axes[1, i].set_title(f'Anomaly Sample {i+1}')

plt.tight_layout()
plt.savefig(f'{DATA_DIR}/sample_data.png', dpi=150, bbox_inches='tight')
plt.show()

## 3. Diffusion Model Training

In [ ]:
if HAS_TF:
    
    class DiffusionModel:
        """
        Complete Diffusion Model for time series.
        """
        
        def __init__(self, seq_length, n_features, T=200):
            self.seq_length = seq_length
            self.n_features = n_features
            self.T = T
            
            # Noise schedule
            self.betas, self.alphas, self.alpha_bar = get_noise_schedule(T, 'cosine')
            
            # Build denoising model
            self.model = build_denoising_model(
                seq_length=seq_length,
                n_features=n_features,
                channels=[32, 64, 32]  # Smaller for faster training
            )
            
            self.optimizer = keras.optimizers.Adam(1e-4)
            
        def forward_diffusion(self, x_0, t):
            """
            Add noise according to schedule.
            
            x_t = √(ᾱ_t) · x_0 + √(1-ᾱ_t) · ε
            """
            noise = tf.random.normal(tf.shape(x_0))
            
            # Get alpha_bar for each sample's timestep
            alpha_bar_t = tf.gather(self.alpha_bar, t)
            alpha_bar_t = tf.reshape(alpha_bar_t, [-1, 1, 1])
            
            # Apply forward diffusion
            x_t = tf.sqrt(alpha_bar_t) * x_0 + tf.sqrt(1 - alpha_bar_t) * noise
            
            return x_t, noise
        
        @tf.function
        def train_step(self, x_0):
            """
            Single training step.
            """
            batch_size = tf.shape(x_0)[0]
            
            # Sample random timesteps
            t = tf.random.uniform([batch_size], 0, self.T, dtype=tf.int32)
            
            # Forward diffusion
            x_t, noise = self.forward_diffusion(x_0, t)
            
            with tf.GradientTape() as tape:
                # Predict noise
                noise_pred = self.model([x_t, t], training=True)
                
                # MSE loss
                loss = tf.reduce_mean((noise - noise_pred) ** 2)
            
            # Update weights
            gradients = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
            
            return loss
        
        def train(self, X, epochs=50, batch_size=32):
            """
            Train the diffusion model.
            """
            dataset = tf.data.Dataset.from_tensor_slices(X.astype(np.float32))
            dataset = dataset.shuffle(len(X)).batch(batch_size)
            
            losses = []
            
            for epoch in range(epochs):
                epoch_loss = 0
                n_batches = 0
                
                for batch in dataset:
                    loss = self.train_step(batch)
                    epoch_loss += loss.numpy()
                    n_batches += 1
                
                avg_loss = epoch_loss / n_batches
                losses.append(avg_loss)
                
                if (epoch + 1) % 10 == 0:
                    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")
            
            return losses
        
        def denoise(self, x_noisy, t):
            """
            Single denoising step.
            """
            noise_pred = self.model([x_noisy, t], training=False)
            
            alpha_t = tf.gather(self.alphas, t)
            alpha_bar_t = tf.gather(self.alpha_bar, t)
            
            alpha_t = tf.reshape(alpha_t, [-1, 1, 1])
            alpha_bar_t = tf.reshape(alpha_bar_t, [-1, 1, 1])
            
            # Predict x_0
            x_0_pred = (x_noisy - tf.sqrt(1 - alpha_bar_t) * noise_pred) / tf.sqrt(alpha_bar_t)
            
            return x_0_pred, noise_pred
        
        def compute_anomaly_score(self, x, n_steps=50):
            """
            Compute anomaly score based on reconstruction error.
            
            1. Add noise at timestep t
            2. Denoise back
            3. Compare to original
            """
            # Use middle timestep for balanced noise
            t = np.full(len(x), n_steps, dtype=np.int32)
            
            x_tf = tf.cast(x, tf.float32)
            t_tf = tf.cast(t, tf.int32)
            
            # Add noise
            x_noisy, _ = self.forward_diffusion(x_tf, t_tf)
            
            # Denoise
            x_recon, _ = self.denoise(x_noisy, t_tf)
            
            # Reconstruction error
            error = tf.reduce_mean((x_tf - x_recon) ** 2, axis=[1, 2])
            
            return error.numpy()
        
        def generate(self, n_samples=10):
            """
            Generate new samples via reverse diffusion.
            """
            # Start from pure noise
            x = tf.random.normal([n_samples, self.seq_length, self.n_features])
            
            # Reverse diffusion
            for t in reversed(range(self.T)):
                t_batch = tf.fill([n_samples], t)
                
                noise_pred = self.model([x, t_batch], training=False)
                
                alpha_t = self.alphas[t]
                alpha_bar_t = self.alpha_bar[t]
                beta_t = self.betas[t]
                
                # Compute mean
                coef1 = 1 / np.sqrt(alpha_t)
                coef2 = beta_t / np.sqrt(1 - alpha_bar_t)
                mean = coef1 * (x - coef2 * noise_pred)
                
                # Add noise (except at t=0)
                if t > 0:
                    noise = tf.random.normal(tf.shape(x))
                    sigma = np.sqrt(beta_t)
                    x = mean + sigma * noise
                else:
                    x = mean
            
            return x.numpy()
    
    print("DiffusionModel class defined")

In [ ]:
if HAS_TF:
    # Normalize data
    scaler = StandardScaler()
    X_normal_flat = X_normal.reshape(-1, X_normal.shape[-1])
    scaler.fit(X_normal_flat)
    
    X_normal_scaled = np.array([scaler.transform(x) for x in X_normal])
    X_anomaly_scaled = np.array([scaler.transform(x) for x in X_anomaly])
    
    # Train on normal data only
    X_train, X_val = train_test_split(X_normal_scaled, test_size=0.15, random_state=42)
    
    print(f"Training samples: {len(X_train)}")
    print(f"Validation samples: {len(X_val)}")
    
    # Build and train model
    diffusion = DiffusionModel(
        seq_length=128,
        n_features=4,
        T=100  # Fewer steps for faster training
    )
    
    diffusion.model.summary()

In [ ]:
if HAS_TF:
    print("Training diffusion model...")
    print("(This may take a few minutes)")
    
    losses = diffusion.train(X_train, epochs=30, batch_size=32)
    
    # Plot training loss
    plt.figure(figsize=(10, 4))
    plt.plot(losses)
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Diffusion Model Training')
    plt.grid(True, alpha=0.3)
    plt.savefig(f'{MODEL_DIR}/training_loss.png', dpi=150, bbox_inches='tight')
    plt.show()

## 4. Anomaly Detection

In [ ]:
if HAS_TF:
    # Compute anomaly scores
    print("Computing anomaly scores...")
    
    scores_normal = diffusion.compute_anomaly_score(X_val, n_steps=30)
    scores_anomaly = diffusion.compute_anomaly_score(X_anomaly_scaled, n_steps=30)
    
    print(f"Normal scores: mean={scores_normal.mean():.4f}, std={scores_normal.std():.4f}")
    print(f"Anomaly scores: mean={scores_anomaly.mean():.4f}, std={scores_anomaly.std():.4f}")

In [ ]:
if HAS_TF:
    # Visualize results
    fig, axes = plt.subplots(1, 3, figsize=(15, 4))
    
    # Distribution of scores
    axes[0].hist(scores_normal, bins=30, alpha=0.7, label='Normal', color='green')
    axes[0].hist(scores_anomaly, bins=30, alpha=0.7, label='Anomaly', color='red')
    axes[0].set_xlabel('Anomaly Score (Reconstruction Error)')
    axes[0].set_ylabel('Count')
    axes[0].set_title('Score Distribution')
    axes[0].legend()
    
    # ROC curve
    labels = np.concatenate([np.zeros(len(scores_normal)), np.ones(len(scores_anomaly))])
    scores = np.concatenate([scores_normal, scores_anomaly])
    
    fpr, tpr, thresholds = roc_curve(labels, scores)
    auc_score = roc_auc_score(labels, scores)
    
    axes[1].plot(fpr, tpr, 'b-', linewidth=2, label=f'AUC = {auc_score:.3f}')
    axes[1].plot([0, 1], [0, 1], 'k--', alpha=0.3)
    axes[1].set_xlabel('False Positive Rate')
    axes[1].set_ylabel('True Positive Rate')
    axes[1].set_title('ROC Curve')
    axes[1].legend()
    
    # Box plot
    axes[2].boxplot([scores_normal, scores_anomaly], labels=['Normal', 'Anomaly'])
    axes[2].set_ylabel('Anomaly Score')
    axes[2].set_title('Score Comparison')
    
    plt.tight_layout()
    plt.savefig(f'{MODEL_DIR}/anomaly_detection_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print(f"\nDiffusion Anomaly Detection AUC: {auc_score:.4f}")

## 5. Synthetic Fault Generation (Data Augmentation)

Use diffusion to generate realistic fault patterns for training classifiers.

In [ ]:
if HAS_TF:
    
    class ConditionalDiffusionModel(DiffusionModel):
        """
        Conditional Diffusion for generating specific fault types.
        """
        
        def __init__(self, seq_length, n_features, n_classes, T=200):
            self.n_classes = n_classes
            super().__init__(seq_length, n_features, T)
            
            # Rebuild with conditioning
            self.model = self._build_conditional_model()
            self.optimizer = keras.optimizers.Adam(1e-4)
            
        def _build_conditional_model(self):
            """
            Build denoising model with class conditioning.
            """
            # Inputs
            x_input = keras.Input(shape=(self.seq_length, self.n_features), name='x_noisy')
            t_input = keras.Input(shape=(), dtype=tf.int32, name='timestep')
            c_input = keras.Input(shape=(), dtype=tf.int32, name='class')  # Class label
            
            # Time embedding
            time_emb = SinusoidalPositionEmbeddings(64)(t_input)
            time_emb = layers.Dense(128, activation='silu')(time_emb)
            time_emb = layers.Dense(64)(time_emb)
            
            # Class embedding
            class_emb = layers.Embedding(self.n_classes, 64)(c_input)
            
            # Combine embeddings
            cond_emb = time_emb + class_emb
            
            # Denoising network
            x = layers.Conv1D(32, 3, padding='same')(x_input)
            
            # Add conditioning via FiLM (Feature-wise Linear Modulation)
            gamma = layers.Dense(32)(cond_emb)
            beta = layers.Dense(32)(cond_emb)
            x = x * gamma[:, None, :] + beta[:, None, :]
            
            x = ResidualBlock1D(64)(x, cond_emb)
            x = ResidualBlock1D(64)(x, cond_emb)
            x = ResidualBlock1D(32)(x, cond_emb)
            
            x = layers.LayerNormalization()(x)
            x = tf.nn.silu(x)
            output = layers.Conv1D(self.n_features, 3, padding='same')(x)
            
            model = keras.Model(inputs=[x_input, t_input, c_input], outputs=output)
            return model
        
        @tf.function
        def train_step_conditional(self, x_0, labels):
            batch_size = tf.shape(x_0)[0]
            t = tf.random.uniform([batch_size], 0, self.T, dtype=tf.int32)
            
            x_t, noise = self.forward_diffusion(x_0, t)
            
            with tf.GradientTape() as tape:
                noise_pred = self.model([x_t, t, labels], training=True)
                loss = tf.reduce_mean((noise - noise_pred) ** 2)
            
            gradients = tape.gradient(loss, self.model.trainable_variables)
            self.optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
            
            return loss
        
        def train_conditional(self, X, labels, epochs=50, batch_size=32):
            dataset = tf.data.Dataset.from_tensor_slices(
                (X.astype(np.float32), labels.astype(np.int32))
            )
            dataset = dataset.shuffle(len(X)).batch(batch_size)
            
            losses = []
            
            for epoch in range(epochs):
                epoch_loss = 0
                n_batches = 0
                
                for x_batch, label_batch in dataset:
                    loss = self.train_step_conditional(x_batch, label_batch)
                    epoch_loss += loss.numpy()
                    n_batches += 1
                
                avg_loss = epoch_loss / n_batches
                losses.append(avg_loss)
                
                if (epoch + 1) % 10 == 0:
                    print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.6f}")
            
            return losses
        
        def generate_class(self, class_label, n_samples=10):
            """
            Generate samples of a specific class.
            """
            x = tf.random.normal([n_samples, self.seq_length, self.n_features])
            labels = tf.fill([n_samples], class_label)
            
            for t in reversed(range(self.T)):
                t_batch = tf.fill([n_samples], t)
                
                noise_pred = self.model([x, t_batch, labels], training=False)
                
                alpha_t = self.alphas[t]
                alpha_bar_t = self.alpha_bar[t]
                beta_t = self.betas[t]
                
                coef1 = 1 / np.sqrt(alpha_t)
                coef2 = beta_t / np.sqrt(1 - alpha_bar_t)
                mean = coef1 * (x - coef2 * noise_pred)
                
                if t > 0:
                    noise = tf.random.normal(tf.shape(x))
                    sigma = np.sqrt(beta_t)
                    x = mean + sigma * noise
                else:
                    x = mean
            
            return x.numpy()
    
    print("ConditionalDiffusionModel defined")

In [ ]:
# Create labeled dataset with different fault types
def generate_labeled_faults(n_per_class=200, seq_length=128, n_features=4):
    """
    Generate labeled data for conditional diffusion.
    """
    X = []
    y = []
    
    fault_types = ['normal', 'spike', 'drift', 'noise_burst', 'degradation']
    
    for class_idx, fault in enumerate(fault_types):
        for _ in range(n_per_class):
            t = np.linspace(0, 4 * np.pi, seq_length)
            features = np.zeros((seq_length, n_features))
            
            # Base signal
            freq = np.random.uniform(0.8, 1.2)
            features[:, 0] = np.sin(freq * t) + np.random.normal(0, 0.1, seq_length)
            features[:, 1] = 0.5 + 0.1 * np.sin(0.2 * t) + np.random.normal(0, 0.02, seq_length)
            features[:, 2] = 0.5 + np.random.normal(0, 0.05, seq_length)
            features[:, 3] = 0.3 * features[:, 0] + 0.5 + np.random.normal(0, 0.08, seq_length)
            
            if fault == 'spike':
                n_spikes = np.random.randint(3, 10)
                spike_locs = np.random.choice(seq_length, n_spikes, replace=False)
                spike_heights = np.random.uniform(2, 5, n_spikes)
                features[spike_locs, 0] += spike_heights
                
            elif fault == 'drift':
                drift_rate = np.random.uniform(0.3, 1.0)
                drift = drift_rate * np.linspace(0, 1, seq_length)
                features[:, 1] += drift
                features[:, 3] += drift * 0.5
                
            elif fault == 'noise_burst':
                burst_start = np.random.randint(0, seq_length // 2)
                burst_len = np.random.randint(20, 50)
                burst_end = min(burst_start + burst_len, seq_length)
                features[burst_start:burst_end, 0] += np.random.normal(0, 1, burst_end - burst_start)
                features[burst_start:burst_end, 2] += np.random.normal(0, 0.5, burst_end - burst_start)
                
            elif fault == 'degradation':
                # Gradual amplitude increase + frequency shift
                degradation = np.linspace(1, 2, seq_length)
                features[:, 0] *= degradation
                freq_shift = np.linspace(0, 0.5, seq_length)
                features[:, 0] += 0.3 * np.sin((freq + freq_shift) * t)
            
            X.append(features)
            y.append(class_idx)
    
    return np.array(X), np.array(y), fault_types

# Generate data
print("Generating labeled fault data...")
X_labeled, y_labeled, fault_types = generate_labeled_faults(n_per_class=150)
print(f"Generated: X={X_labeled.shape}, y={y_labeled.shape}")
print(f"Classes: {fault_types}")

In [ ]:
if HAS_TF:
    # Normalize
    scaler_cond = StandardScaler()
    X_labeled_flat = X_labeled.reshape(-1, X_labeled.shape[-1])
    scaler_cond.fit(X_labeled_flat)
    
    X_labeled_scaled = np.array([scaler_cond.transform(x) for x in X_labeled])
    
    # Build and train conditional model
    cond_diffusion = ConditionalDiffusionModel(
        seq_length=128,
        n_features=4,
        n_classes=len(fault_types),
        T=100
    )
    
    print("Training conditional diffusion model...")
    cond_losses = cond_diffusion.train_conditional(
        X_labeled_scaled, y_labeled, 
        epochs=30, 
        batch_size=32
    )

In [ ]:
if HAS_TF:
    # Generate synthetic samples for each class
    print("\nGenerating synthetic samples...")
    
    fig, axes = plt.subplots(len(fault_types), 2, figsize=(14, 3 * len(fault_types)))
    
    for class_idx, fault_name in enumerate(fault_types):
        # Real sample
        real_idx = np.where(y_labeled == class_idx)[0][0]
        real_sample = X_labeled_scaled[real_idx]
        
        # Generated sample
        generated = cond_diffusion.generate_class(class_idx, n_samples=1)[0]
        
        # Plot real
        axes[class_idx, 0].plot(real_sample[:, 0], label='Vibration', alpha=0.7)
        axes[class_idx, 0].plot(real_sample[:, 1], label='Temperature', alpha=0.7)
        axes[class_idx, 0].set_title(f'{fault_name} - Real')
        axes[class_idx, 0].set_ylabel('Value')
        if class_idx == 0:
            axes[class_idx, 0].legend()
        
        # Plot generated
        axes[class_idx, 1].plot(generated[:, 0], label='Vibration', alpha=0.7)
        axes[class_idx, 1].plot(generated[:, 1], label='Temperature', alpha=0.7)
        axes[class_idx, 1].set_title(f'{fault_name} - Generated')
    
    axes[-1, 0].set_xlabel('Time Step')
    axes[-1, 1].set_xlabel('Time Step')
    
    plt.tight_layout()
    plt.savefig(f'{MODEL_DIR}/synthetic_faults.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    print("\nSynthetic fault patterns generated!")
    print("These can be used to augment training data for classifiers.")

## 6. Time Series Imputation

Use diffusion to fill missing sensor values.

In [ ]:
if HAS_TF:
    
    def impute_missing_values(diffusion_model, x_with_missing, mask, n_iterations=10):
        """
        Impute missing values using diffusion.
        
        Args:
            x_with_missing: Data with missing values (can be any value where mask=0)
            mask: Binary mask (1=observed, 0=missing)
            n_iterations: Number of refinement iterations
        """
        x = tf.cast(x_with_missing, tf.float32)
        mask = tf.cast(mask, tf.float32)
        
        for i in range(n_iterations):
            # Add small noise
            t = 20  # Low noise level
            t_batch = tf.fill([len(x)], t)
            
            # Only add noise to missing regions
            noise = tf.random.normal(tf.shape(x)) * 0.1
            x_noisy = x + noise * (1 - mask)
            
            # Denoise
            x_denoised, _ = diffusion_model.denoise(x_noisy, t_batch)
            
            # Keep observed values, update missing
            x = mask * x + (1 - mask) * x_denoised
        
        return x.numpy()
    
    # Create data with missing values
    test_sample = X_normal_scaled[0:5].copy()
    
    # Create missing mask (simulate sensor dropout)
    mask = np.ones_like(test_sample)
    
    for i in range(len(test_sample)):
        # Random missing segments
        n_missing = np.random.randint(2, 5)
        for _ in range(n_missing):
            start = np.random.randint(0, 100)
            length = np.random.randint(10, 30)
            feature = np.random.randint(0, 4)
            mask[i, start:start+length, feature] = 0
    
    # Zero out missing values
    test_with_missing = test_sample * mask
    
    # Impute
    print("Imputing missing values...")
    imputed = impute_missing_values(diffusion, test_with_missing, mask, n_iterations=20)

In [ ]:
if HAS_TF:
    # Visualize imputation
    fig, axes = plt.subplots(3, 1, figsize=(14, 8))
    
    sample_idx = 0
    feature_idx = 0
    
    # Original
    axes[0].plot(test_sample[sample_idx, :, feature_idx], 'b-', linewidth=2)
    axes[0].set_title('Original Signal')
    axes[0].set_ylabel('Value')
    
    # With missing (show gaps)
    missing_signal = test_with_missing[sample_idx, :, feature_idx].copy()
    missing_signal[mask[sample_idx, :, feature_idx] == 0] = np.nan
    axes[1].plot(missing_signal, 'b-', linewidth=2)
    # Highlight missing regions
    missing_regions = np.where(mask[sample_idx, :, feature_idx] == 0)[0]
    if len(missing_regions) > 0:
        axes[1].fill_between(range(len(missing_signal)), 
                            axes[1].get_ylim()[0], axes[1].get_ylim()[1],
                            where=(mask[sample_idx, :, feature_idx] == 0),
                            alpha=0.3, color='red', label='Missing')
    axes[1].set_title('Signal with Missing Data')
    axes[1].set_ylabel('Value')
    axes[1].legend()
    
    # Imputed
    axes[2].plot(test_sample[sample_idx, :, feature_idx], 'b-', linewidth=2, label='Original', alpha=0.5)
    axes[2].plot(imputed[sample_idx, :, feature_idx], 'g--', linewidth=2, label='Imputed')
    axes[2].set_title('Imputed Signal')
    axes[2].set_xlabel('Time Step')
    axes[2].set_ylabel('Value')
    axes[2].legend()
    
    plt.tight_layout()
    plt.savefig(f'{MODEL_DIR}/imputation_results.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Compute imputation error
    missing_mask = (mask == 0)
    if missing_mask.sum() > 0:
        imputation_error = np.abs(test_sample[missing_mask] - imputed[missing_mask]).mean()
        print(f"\nMean Imputation Error (on missing values): {imputation_error:.4f}")

## 7. Save Models

In [ ]:
if HAS_TF:
    # Save models
    diffusion.model.save(f'{MODEL_DIR}/diffusion_anomaly_detector.keras')
    cond_diffusion.model.save(f'{MODEL_DIR}/conditional_diffusion.keras')
    
    # Save noise schedule
    np.savez(
        f'{MODEL_DIR}/noise_schedule.npz',
        betas=diffusion.betas,
        alphas=diffusion.alphas,
        alpha_bar=diffusion.alpha_bar
    )
    
    # Save metadata
    metadata = {
        'model_type': 'Denoising Diffusion Probabilistic Model',
        'models': {
            'anomaly_detector': {
                'file': 'diffusion_anomaly_detector.keras',
                'T': diffusion.T,
                'seq_length': diffusion.seq_length,
                'n_features': diffusion.n_features,
                'auc': float(auc_score)
            },
            'conditional': {
                'file': 'conditional_diffusion.keras',
                'classes': fault_types
            }
        },
        'applications': [
            'Anomaly detection via reconstruction error',
            'Synthetic fault generation for data augmentation',
            'Time series imputation for missing values'
        ],
        'advantages': [
            'High-quality synthetic data generation',
            'Works with limited labeled data',
            'Probabilistic uncertainty quantification'
        ]
    }
    
    with open(f'{MODEL_DIR}/diffusion_metadata.json', 'w') as f:
        json.dump(metadata, f, indent=2)
    
    print(f"\nModels saved to {MODEL_DIR}/")

## 8. Node-RED Integration

In [ ]:
node_red_code = '''
// Node-RED Function: Diffusion-based Anomaly Detection

const SEQ_LENGTH = 128;
const N_FEATURES = 4;
const NOISE_STEP = 30;  // How much noise to add for reconstruction

// Collect sensor readings
if (!context.buffer) {
    context.buffer = [];
}

context.buffer.push([
    msg.payload.vibration,
    msg.payload.temperature,
    msg.payload.pressure,
    msg.payload.current
]);

if (context.buffer.length > SEQ_LENGTH) {
    context.buffer.shift();
}

if (context.buffer.length < SEQ_LENGTH) {
    msg.payload = { status: "collecting", samples: context.buffer.length };
    return msg;
}

// Prepare for model
// The diffusion model will:
// 1. Add noise to the input
// 2. Attempt to denoise
// 3. Compare reconstruction to original

msg.payload = {
    input: [context.buffer],
    noise_step: NOISE_STEP,
    model: "diffusion_anomaly",
    
    // Thresholds for alerting
    warning_threshold: 0.5,
    critical_threshold: 1.0
};

return msg;
''';

print("Node-RED Integration Code:")
print("=" * 50)
print(node_red_code)

## Summary

This notebook demonstrated **Diffusion Models** for Predictive Maintenance:

### Key Concepts:

| Component | Purpose |
|-----------|--------|
| **Forward Diffusion** | Gradually add noise to data |
| **Reverse Diffusion** | Learn to denoise step by step |
| **Anomaly Score** | Reconstruction error indicates anomalies |
| **Conditional Generation** | Generate specific fault types |

### Applications:

1. **Anomaly Detection** - High reconstruction error = anomaly
2. **Data Augmentation** - Generate synthetic faults for training
3. **Imputation** - Fill missing sensor values
4. **Uncertainty Quantification** - Probabilistic predictions

### When to Use Diffusion Models:

- Limited fault data (few-shot learning)
- Need to generate realistic synthetic data
- Want probabilistic anomaly scores
- Dealing with missing or corrupted sensor data